In [1]:
#  ----------------------------------  using python wraper -------------------
#  ---------------------------------- Scrape included word -------------------
import os
import pandas as pd
import snscrape.modules.twitter as twitter
import json
def scrape_tweet_data(username):
    
    DATASET_COLUMNS=['tweetDate', 'userId','userName', 'tweetId', 'tweetText', 'retweetedTweet', 'quotedTweet', 'hashtag', 'replyCount', 'retweetCount', 'likeCount', 'viewCount']
    attributes_container = []
    tweet_count = 5
    # text_query = "Shahrukh Khan"
    # since_date = "2023-01-01"
    # until_date = "2023-02-15"
    # for i, tweet in enumerate(twitter.TwitterProfileScraper('iamsrk').get_items()):
    for i, tweet in enumerate(twitter.TwitterUserScraper(username).get_items()):
    # for i, tweet in enumerate(twitter.TwitterSearchScraper(text_query + ' since:'+since_date+' until:'+until_date+' lang:"en" ').get_items()):
        if(i == tweet_count):
            break
        print(tweet)
        # print(type(tweet.quotedTweet) )
        qt = tweet.quotedTweet
        # print(qt)
        if(qt is not None):
            # d_list = json.loads(qt)
            # snscrape.modules.twitter.Tweet qtweet = qt
            print(qt.id)
        # print d_list.get('id')
        rt = tweet.retweetedTweet
        # print(rt)
        attributes_container.append([tweet.date,tweet.user.id,tweet.user.username,str(tweet.id),tweet.rawContent,rt,qt,tweet.hashtags, tweet.replyCount, tweet.retweetCount, tweet.likeCount,tweet.viewCount])
    tweets_df = pd.DataFrame(attributes_container, columns=DATASET_COLUMNS)
    save_as = str(username)+"_tweets.csv"
    tweets_df.to_csv(save_as, index=False)
    file_path = os.path.abspath(save_as)
    print("=> data saved as CSV at lcoation: " + file_path)
    return file_path

In [ ]:
#  ----------------------------------  using python wraper -------------------
#  ---------------------------------- Scrape included word -------------------
import os
import snscrape.modules.instagram as sninstagram
import pandas as pd
import numpy as np
import json
data = pd.DataFrame(sninstagram.InstagramUserScraper('shahrukh').get_items())
data.head()
    
    


In [ ]:
import snscrape.modules.instagram as sninstagram
import pandas as pd

query='shahrukh' #change name
ins_s=[]
limit=10
for ins in sninstagram.InstagramHashtagScraper(query).get_items():
      print(vars(ins))
      break

In [1]:
import os
import pandas as pd
import json
def get_tweet_by_date(username,since_date,until_date,max_count):
    os.system("snscrape --jsonl --max-results {} --since {} twitter-search 'from:{} until:{}'> {}--tweets.json".format(max_count,since_date,username,until_date,username))
    # Reads the json generated from the CLI command above and creates a pandas dataframe
    tweets_df = pd.read_json('{}-tweets.json'.format(username), lines=True)
    DATASET_COLUMNS=['date', 'id', 'rawContent', 'retweetedTweet', 'quotedTweet', 'hashtags', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'viewCount']
    new_df = tweets_df[DATASET_COLUMNS].copy()
    return new_df


In [3]:
import json
def fix_nonascii(input_str):
    # Convert the string to valid Unicode escape sequences
    escaped_str = input_str.encode('unicode_escape').decode('utf-8')
    # Remove non-ASCII characters
    ascii_str = escaped_str.encode('ascii', 'ignore').decode('utf-8')
    # Convert the string back to its original form
    fixed_str = ascii_str.encode('utf-8').decode('unicode_escape')
    return fixed_str

In [4]:
import re
replaced = re.sub(r'[^\x00-\x7f]',r'', "Find the topic and sentiment from the following social media content.\n\n@Chad_Hurley \ud83d\udc40\n") 
print(replaced)

Find the topic and sentiment from the following social media content.

@Chad_Hurley 



In [5]:
import nltk
import string
import re
from nltk.corpus import stopwords
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = str(text).lower()
    text = text.replace('@', 'at')
    pat = r"[{}]".format(string.punctuation) 
    text = re.sub(pat, ' ', text) 
    text = re.sub('  +', ' ', text) 
    text = re.sub('https?://|www\.', '', text)  
    text = re.sub('\n', '', text)
    # text = re.sub('\w*\d\w*', '', text) 
    # print(text)
    # text = text.replace(u'\ud800', '')
    # text = text.replace('\x80', '?')
    # text = text.translate({i: None for i in range(128, 256)})
    # fixed_text = fix_nonascii(text)
    text = re.sub(r'[^\x00-\x7f]',r'',text)
    # text = fixed_text
    # print(text)
    
    # remove stopwords
    text = ' '.join([word for word in text.split() if word not in stop_words])
    if len(text) == 0:
        text = " "
    return text

In [ ]:
import os
import openai
openai.organization = "org-PcoGruCs6IkfEDRsFbzrDtGA"
openai.api_key = os.getenv("OPENAI_API_KEY")
# models = openai.Model.list()

# for model in models["data"]:
#     print(model["id"])
#     # print(model)
def getModerationOutput(inputText):
    response = openai.Moderation.create(
      input=inputText,
    )
    # convert the OpenAIObject into a JSON string
    response_json_str = json.dumps(response)
    # parse the JSON response into a dictionary
    response_dict = json.loads(response_json_str)
    # print(response_dict)
    # get the categories dictionary from the first result
    categories_dict = response_dict['results'][0]['categories']
    isNegative = False
    for category, value in categories_dict.items():
        # print(category, value)
        isNegative = value
    # print the categories
    # print(categories_dict)
    # result = 'Positive'
    # if isNegative:
    #     result = 'Negative'
    # return result
    return 'Negative' if isNegative else 'Positive'

In [ ]:
def extract_topic_using_gpt(sentence):
        openai.api_key = os.getenv("OPENAI_API_KEY")
        pormpt_prefix = "Find the topic and sentiment(positive, negative, nutral) from the following social media content.\n\n"
        input_sentence = pormpt_prefix + sentence
        response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=input_sentence,
        max_tokens=560,
        top_p=1,
        n=1,
        stop=None,
        temperature=1,
        frequency_penalty=0,
        presence_penalty=0
        )
        message = response["choices"][0]["text"]
        keyword = "Topic:"
        start_index = message.find(keyword)
        substring = message[start_index:]
        # print(substring)
        return substring;

In [ ]:
import pandas as pd
from tabulate import tabulate
def showTabulatedDf(df):
    # set column width to wrap text
    pd.set_option('display.max_colwidth', 10)
    
    # format dataframe as a table with wrapped text
    table = tabulate(df, headers='keys', tablefmt='pipe')
    
    # print the table
    print(table)

In [ ]:
import os
import pandas as pd
import time
from datetime import datetime

username = "ElonMusk"
since_date = "2023-01-01"
until_date = "2023-05-29"
max_count = 10000


# Get current date and time
now = datetime.now()

# Format current date as string
date_string = now.strftime("%Y-%m-%d")

print(date_string)  # Output: 2023-04-15


# Calculate the delay based on your rate limit
# rate_limit_per_minute = 20
# delay = 60.0 / rate_limit_per_minute

 
# def consumeModerationApi(x):
#     # wait until there are enough tokens in the bucket
#     # while not bucket.consume(1):
#     #     time.sleep(5)
#      # Sleep for the delay
#     time.sleep(delay)
#     # make the request
#     print(f'making getModerationOutput request with data {x}')
#     return getModerationOutput(x)

# def consumeCompletionApi(x):
#     # wait until there are enough tokens in the bucket
#     # while not bucket.consume(1):
#     #     time.sleep(5)
#      # Sleep for the delay
#     time.sleep(delay)
#     # make the request
#     print(f'making extract_topic_using_gpt request with data {x}')
#     return extract_topic_using_gpt(x)


tweet_df = get_tweet_by_date(username,since_date,until_date,max_count)
print(tweet_df.shape)
# print('Missing values percentage')
# print(tweet_df.isnull().sum() * 100 / len(tweet_df))
print('Missing values counts')
print(tweet_df.isnull().sum())
# remove stop words
tweet_df['clean'] = tweet_df['rawContent'].apply(lambda row: clean_text(row))
# print(tweet_df.head())
tweet_df.to_csv('{}_tweets_{}.csv'.format(username,date_string), index=False)
# showTabulatedDf(tweet_df)
# tweet_df['sentiment'] = tweet_df['clean'].apply(lambda x: consumeModerationApi(x))
# %time tweet_df['sentimentRaw'] = tweet_df['rawContent'].apply(lambda x: consumeModerationApi(x))
# get the topic
# %time tweet_df['topic'] = tweet_df['rawContent'].apply(lambda x: consumeCompletionApi(x))
# tweet_df['topicClean'] = tweet_df['clean'].apply(lambda x: consumeCompletionApi(x))
# tweet_df.head()
# df_selected = tweet_df.loc[:, ['rawContent','topic']]
# df_selected = tweet_df.loc[:, ['rawContent','topic','sentimentRaw']]
# df_selected2 = tweet_df.loc[:, ['clean','topicClean','sentiment']]
# print(df_selected.head())
# df_selected.head()
# showTabulatedDf(df_selected)
# showTabulatedDf(df_selected2)
# tweet_df.to_csv('{}-tweets.csv'.format(username), index=False)

In [ ]:
import json
import pandas as pd
df = pd.read_csv('iamsrk_tweets_2023-04-17.csv')
filename = "requests_to_parallel_process_iamsrk_2023-04-17_sentiment.jsonl"
n_requests = 10000
# pormpt_prefix = "Find the topic and sentiment(positive, negative, nutral) from the following social media content.\n\n"
# pormpt_prefix = "What are the three most relevant action words that describe the main idea, sentiment, and any mentioned name in this tweet?.\n"
pormpt_prefix = "What is the single action words that describe the sentiment in this tweet?.If you find nothing answer neutral?.\n"
# prompt = f"Tweet: \"{}\"\n\nPrompt: What is the main idea, sentiment and any mention of a name in the tweet?\n\nOutput:\n\nIdea: \nSentiment: \nName: "
# prompt = f"Twitter post: \"{}\"
# \nPrompt: Summarize, Evaluate, Identify\nMain Idea: Summarize\nSentiment: Evaluate\nName: Identify"
# input_sentence = pormpt_prefix + sentence
# jobs = [{"model": "text-davinci-003", "input": str(x) + "\n"} for x in range(n_requests)]
# jobs = df['rawContent'].apply(lambda row: {"model": "text-embedding-ada-002", "prompt": str(pormpt_prefix+row) + "\n"})
jobs = df['clean'].apply(lambda row: {"model": "text-davinci-003", "prompt": str(pormpt_prefix+row) + "\n"})
# jobs = df['clean'].apply(lambda row: {"model": "text-davinci-003", "prompt": f"Tweet: \"{row}\"\n\nPrompt: What is the main idea, sentiment and any mention of a name in the tweet?\n\nOutput:\n\nIdea: \nSentiment: \nName: "})
count = 0
with open(filename, "w") as f:
    for job in jobs:
        # job = job.replace('\x80', '?')
        count = count + 1
        json_string = json.dumps(job)
        f.write(json_string + "\n")
        if(count==n_requests):
            break

In [ ]:
import json
import pandas as pd
# ---
with open('news_catcher_data_full.json', 'r') as file:
    # Load the JSON data from the file
    data = json.load(file)
    # data = json.loads(json_data)
    # Get a list of titles from the articles
    titles = []
    for article in data['articles']:
        titles.append(article['title'])
    # Print the list of titles
    print(len(titles))

filename = "requests_to_parallel_process_news_full.jsonl"
# n_requests = 10_000
pormpt_prefix = "Find the main topic from the following news headline title.\n\n"
with open(filename, "w") as f:
    for title in titles:
        # job = job.replace('\x80', '?')
        job = {"model": "text-davinci-003", "prompt": str(pormpt_prefix+title) + "\n"}
        json_string = json.dumps(job)
        f.write(json_string + "\n")

In [ ]:
def getCleanText(content):
    # content = "Find the topic and sentiment from the following social media content.\n\nlucky teachers professors teach us fun us also educational rockstars https co\n!"

    start_index = content.find('\n\n') + 2 # Find the index of the first occurrence of "\n\n" and add 2 to get the start index of the desired text
    end_index = content.rfind('\n') # Find the index of the last occurrence of "\n"
    extracted_text = "Couldn't extract text"
    if start_index != -1 and end_index != -1:
        extracted_text = content[start_index:end_index] # Extract the text between start_index and end_index
        # print(extracted_text)
    else:
        print(extracted_text)
    return extracted_text    
        

In [ ]:
import json

def getTopicAndSentiment(content):
    
    list_of_dicts = content
    for d in list_of_dicts:
        for k, v in d.items():
            if isinstance(v, str):
                d[k] = v.replace("'", '"')
    json_str = json.dumps(list_of_dicts)
    
    json_obj = json.loads(json_str) # Parse the JSON string into a Python object
    resultArray = []
    if len(json_obj) > 0:
        text = json_obj[0]['text'] # Access the value of the 'text' key in the first element of the list
        # text = text.strip()
        lines = text.split('\n') # Split the text into lines using '\n' as the separator
        # print(f'json_obj-0-text == {text}')
        # print(len(lines))
        # print(lines)
        topic = 'none'
        sentiment = 'undefined'
        if len(lines) > 1:
            topicLines = lines[1].split(': ')
            if len(topicLines) > 1:
                topic = topicLines[1]  # Extract the topic from the second line of the text
            else:
                topic = lines[1]
            # topic = lines[1].split(': ')[1] # Extract the topic from the second line of the text
        if len(lines) > 2:
            sentimentLines = lines[2].split(': ')
            if len(sentimentLines) > 1:
                sentiment = sentimentLines[1]  # Extract the sentiment from the third line of the text
            # sentiment = lines[2].split(': ')[1] # Extract the sentiment from the third line of the text
        # print('Topic==', topic)
        # print('Sentiment==', sentiment)
        resultArray.extend([topic,sentiment])
    else:
        print('JSON object is empty')
    return resultArray

In [ ]:
import json
import pandas as pd
username = "elonmusk"
# username = "iamsrk"
clean_data = []
topics = []
sentiments =[]
with open(f'parallel_process_results_elonmusk_2023-04-15_100_sentiment.jsonl','r') as file:
    for line in file.__iter__():
        text = line.strip()
        raw_json = json.loads(text)
        # print(raw_json)
        df = pd.DataFrame(raw_json)
        # print(df.shape)
        # showTabulatedDf(df)
#         extract clean data
        clean = df['prompt'][0]
        clean = getCleanText(clean)
        clean_data.append(clean)
        # print(clean_data)
#         extract topic and sentiment
        topic = df['choices'][1]
        topicSentiment = getTopicAndSentiment(topic)
        topics.append(topicSentiment[0])
        sentiments.append(topicSentiment[1])
        # print(topics)
        # print(sentiments)
data = {'clean':clean_data,'topic':topics, 'sentiment':sentiments}
df2 = pd.DataFrame(data) 
# showTabulatedDf(df2)
print('clean-topic-sentiment df')
print(df2.shape)
# jobs = df['clean'].apply(lambda row: {"model": "text-davinci-003", "prompt": str(pormpt_prefix+row) + "\n"})
# ra/gpt_model/elonmusk__tweets.csv
df1 = pd.read_csv('elonmusk_tweets_2023-04-15.csv')
print(df1.shape)
# showTabulatedDf(df1)

merged_df = pd.merge(df1,df2, left_on='clean',right_on='clean',how='right')
print(merged_df.head())
# df['clean'] = tweet_df['rawContent'].apply(lambda row: clean_text(row))
# merged_df.to_csv('raw_tweet_data/{}-tweets-with-topics-sentiment.csv'.format(username), index=False)
unique_merged_df = merged_df.drop_duplicates(subset=['id','clean'])
# unique_merged_df.to_csv('raw_tweet_data/{}-tweets-with-topics-sentiment.csv'.format(username), index=False)
print('merged-df')
print(merged_df.shape)
print('unique_merged_df')
print(unique_merged_df.shape)

In [ ]:
import spacy
# !python -m spacy download en_core_web_sm
nlp = spacy.load('en_core_web_sm')

In [ ]:
# tweet + topic + sentiment + name
import json
import pandas as pd
import spacy
import re
nlp = spacy.load("en_core_web_sm")
# username = "elonmusk"
username = "iamsrk"
clean_data = []
topics = []
sentiments =[]
names = []
count = 0
with open(f'parallel_process_results_iamsrk_2023-04-17_p1.jsonl','r') as file:
    for line in file.__iter__():
        # count = count +1
        # if(count == 6):
        #     break
        text = line.strip()
        raw_json = json.loads(text)

        df = pd.DataFrame(raw_json)
        sentence = df['prompt'][0]
        match = re.search(r'\n(.*?)\n', sentence)
        if match:
            extracted_text = match.group(1)
        clean = extracted_text
        # print(clean)
        clean_data.append(clean)
#        extract topic and sentiment
        topic = df['choices'][1]
        topicJson = topic[0]
        sentence = topicJson['text']
        # print(sentence)
        # input pattern
        pattern = sentence
        # extract words that start with "at" or "At"
        words = re.findall(r"\b(?:a|A)t\w+", pattern)
        name = ''
        if len(words) > 0:
            name = words[0][2:]
            # print(name)
        sentence = sentence.replace("Sentiment:", "").replace("Name:", "")
        doc = nlp(sentence)
        # print(doc)
        # extract nouns, verbs, adjectives and adverbs
        # words = [token.text for token in doc if token.pos_ in ['NOUN', 'VERB', 'ADJ', 'ADV']]
        words = [token.text for token in doc if not token.is_space and token.is_alpha]
        # main_words.append(words)
        # print(words)
        topics.append(words)
        names.append(name)

data3 = {'clean':clean_data,'topic':topics, 'names':names}
df3 = pd.DataFrame(data3) 
print('clean-topic-name df')
print(df3.shape)
print(df3.head())

count = 0;
clean_data = []
sentiments =[]
with open(f'parallel_process_results_iamsrk_2023-04-17_sentiment.jsonl','r') as file:
    for line in file.__iter__():
        # count = count +1
        # if(count == 6):
        #     break
        text = line.strip()
        raw_json = json.loads(text)
        df = pd.DataFrame(raw_json)
        sentence = df['prompt'][0]
        # print(sentence)
        match = re.search(r'\n(.*?)\n', sentence)
        if match:
            extracted_text = match.group(1)
        clean = extracted_text
        # print(clean)
        clean_data.append(clean)
#        extract topic and sentiment
        topic = df['choices'][1]
        topicJson = topic[0]
        sentence = topicJson['text']
        # print(sentence)
        doc = nlp(sentence)
        words = [token.text for token in doc if not token.is_space and token.is_alpha]
        sentiments.append(words)
        
data2 = {'clean':clean_data,'sentiment':sentiments}
df2 = pd.DataFrame(data2) 
print('clean--sentiment df')
print(df2.shape)
print(df2.head())

df1 = pd.read_csv('iamsrk_tweets_2023-04-17.csv')
print(df1.shape)
print(df1.head())

merged_df = pd.merge(df1,df2, left_on='clean',right_on='clean',how='right')
print('1st merged-df')
print(merged_df.shape)
print(merged_df.head())
merged_df = pd.merge(merged_df,df3, left_on='clean',right_on='clean',how='right')
print('2nd merged-df')
print(merged_df.shape)
print(merged_df.head())
# df['clean'] = tweet_df['rawContent'].apply(lambda row: clean_text(row))
# merged_df.to_csv('raw_tweet_data/{}-tweets-with-topics-sentiment.csv'.format(username), index=False)
unique_merged_df = merged_df.drop_duplicates(subset=['id','clean'])
# unique_merged_df.to_csv('raw_tweet_data/{}-tweets-with-topics-sentiment.csv'.format(username), index=False)
print('unique_merged_df')
print(unique_merged_df.shape)
print(unique_merged_df.head())
unique_merged_df.to_csv('raw_tweet_data/{}-tweets-with-topics-sentiment-name-2023-04-15.csv'.format(username), index=False)

In [ ]:
import json
import pandas as pd
clean_data = []
topics = []
with open(f'raw_tweet_data/requests_to_parallel_process_results_news_full.jsonl','r') as file:
    for line in file.__iter__():
        text = line.strip()
        raw_json = json.loads(text)
        # print(raw_json)
        df = pd.DataFrame(raw_json)
        # print(df.shape)
        # showTabulatedDf(df)
#         extract clean data
        clean = df['prompt'][0]
        clean = getCleanText(clean)
        clean_data.append(clean)
        # print(clean_data)
#         extract topic and sentiment
        topic = df['choices'][1]
        topicSentiment = getTopicAndSentiment(topic)
        topics.append(topicSentiment[0])
        # print(topics)
        # print(sentiments)
data = {'topic':topics}
df = pd.DataFrame(data) 
# showTabulatedDf(df1)
print('topic df')
print(df.shape)
df.to_csv('raw_tweet_data/news_topics_full.csv', index=False)

In [ ]:
import asyncio
asyncio.run(
        process_api_requests_from_file(
            requests_filepath="requests_to_parallel_process.jsonl",
            save_filepath="requests_to_parallel_process_result.jsonl",
            request_url="https://api.openai.com/v1/embeddings",
            api_key=os.getenv("OPENAI_API_KEY"),
            max_requests_per_minute=float(20),
            max_tokens_per_minute=float(150000),
            token_encoding_name="cl100k_base",
            max_attempts=int(5),
            logging_level=int(20),
        )
 )

In [ ]:
import tokenizers

# Load the BPE tokenizer with a vocabulary size of 50,000
tokenizer = tokenizers.ByteLevelBPETokenizer(vocab_file=None, add_prefix_space=True, num_threads=1, 
                                              min_frequency=2, lowercase=True)
tokenizer.train(files, vocab_size=50000)

# Get the BPE encoding for a given text
text = "This is a sample text to encode using BPE."
encoding = tokenizer.encode(text)

# Print the encoded tokens
print(encoding.tokens)

In [25]:
# topic classification
# topic / word - sentiment
#  ra/gpt_model/raw_tweet_data/iamsrk-tweets-with-topics-sentiment-name-2023-04-15.csv
import pandas as pd
import ast
import re
df_original = pd.read_csv('raw_tweet_data/iamsrk-tweets-with-topics-sentiment-name-2023-04-15.csv')
# print(df_original.shape)
# print(df_original.head())

# df = pd.DataFrame({
#     'sentiment': ["['Neutral']", "['Neutral']", "['Neutral']"],
#     'topic': ["['Go', 'Wrong', 'AI']", "['Success', 'Excitement', 'Guaranteed']", "['Take', 'Walk', 'Outrageous']"]
# })
# new_df = pd.DataFrame(columns=['sentiment', 'topic'])
df =  df_original.loc[:, ['topic', 'sentiment']]

# split the values in the topic column and expand them into separate rows
df2 = df.assign(topic=df['topic'].str.split(', ')).explode('topic').reset_index(drop=True)

# remove the square brackets from the sentiment and topic values
df2['sentiment'] = df2['sentiment'].str.replace('[', '').str.replace(']', '').str.replace("'","")
df2['word'] = df2['topic'].str.replace('[', '').str.replace(']', '').str.replace("'","")

print(df2.shape)
# df2 = df2.drop_duplicates()
df2 = df2.drop_duplicates(subset=['word'])
df2 =  df2.loc[:, ['word', 'sentiment']]
print(df2.shape)
print(df2.head(10))
df2.to_csv('raw_tweet_data/iamsrk-topic-classification-2023-04-15-with-sentiment.csv', index=False)
# Print the new data frame
# print(new_df)
# Print the new DataFrame
# print(new_df)

# Print the resulting dataframe
# print(new_df.head(10))

# convert string to list of strings
df_original['topic'] = df_original['topic'].apply(lambda x: ast.literal_eval(x))

# create a new data frame with a single column for the individual words
df_new = pd.DataFrame({'word': [word for row in df_original['topic'] for word in row]})
# drop duplicate rows
print(df_new.shape)
df_new = df_new.drop_duplicates()
print(df_new.shape)
# print(df_new.head())
# df_new.to_csv('raw_tweet_data/elonmusk-topic-classification-2023-04-15.csv', index=False)

(1217, 3)
(605, 2)
               word       sentiment
0          Remember         Believe
1           Believe         Believe
2   Congratulations         Believe
4        Maybelline  Congratulatory
5            Credit  Congratulatory
6             Watch           Watch
7                At           Watch
9          Download        Download
10       Knightclub        Download
11              App        Download
(1189, 1)
(604, 1)


/var/folders/lv/44h1yhq55ld6x4th27m94n4c0000gn/T/ipykernel_18249/2324336251.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['sentiment'] = df2['sentiment'].str.replace('[', '').str.replace(']', '').str.replace("'","")
/var/folders/lv/44h1yhq55ld6x4th27m94n4c0000gn/T/ipykernel_18249/2324336251.py:21: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['word'] = df2['topic'].str.replace('[', '').str.replace(']', '').str.replace("'","")


In [40]:
# topic classification
# topic / word - sentiment -- from merging prompt, output -- prompt -- output
#  ra/gpt_model/raw_tweet_data/iamsrk-tweets-with-topics-sentiment-name-2023-04-15.csv
import pandas as pd
import ast
import re
def merge_post_word_sentiment(path_csv_sentiment,path_csv_topic,path_merged_post_word_sentiment):
    df2 = pd.read_csv(path_csv_sentiment) 
    print('post--sentiment DF')
    print(df2.shape)
    # print(df2.head(1))

    df1 = pd.read_csv(path_csv_topic)
    print('post--topic DF')
    print(df1.shape)
    # print(df1.head(1))

    merged_df = pd.merge(df1,df2, left_on='Prompt',right_on='Prompt',how='right')
    print('merged-df')
    print(merged_df.shape)
    # print(merged_df.head(1))
    merged_df.dropna(subset=['Output_x'], inplace=True)
    df_final = pd.DataFrame()
    df_final['post'] = merged_df['Prompt']
    df_final['word'] = merged_df['Output_x'] 
    df_final['sentiment'] = merged_df['Output_y']
    
        # split the values in the topic column and expand them into separate rows
    df2 = df_final.assign(word=df_final['word'].str.split(', ')).explode('word').reset_index(drop=True)
    
        # remove the square brackets from the sentiment and topic values
    df2['sentiment'] = df2['sentiment'].str.replace('[', '').str.replace(']', '').str.replace("'","")
    df2['word'] = df2['word'].str.replace('[', '').str.replace(']', '').str.replace("'","")
    
    # df2 = df2.drop_duplicates()
    # df2 = df2.dropna()
    # print('df_final-df')
    print(df2.head(10))
    
    df2.to_csv(path_merged_post_word_sentiment, index=False)
    
p_csv_sentiment = 'parsed/result_narendramodi_sentiment.csv'
p_csv_topic = 'parsed/result_narendramodi_topic.csv'
p_csv_post_topic_sentiment = 'parsed/modi_post_topic_sentiment.csv'
merge_post_word_sentiment(p_csv_sentiment,p_csv_topic,p_csv_post_topic_sentiment)

post--sentiment DF
(100, 2)
post--topic DF
(99, 2)
merged-df
(122, 3)
                    post         word  sentiment
0    https co i3pdwd5gl7        love,   positive
1    https co i3pdwd5gl7       happy,   positive
2    https co i3pdwd5gl7     awesome.   positive
3    https co j1tx5obii2      follow,   positive
4    https co j1tx5obii2        like,   positive
5    https co j1tx5obii2     retweet.   positive
6  9 https co akfkghxhnz        link,   positive
7  9 https co akfkghxhnz  followed by   positive
8  9 https co akfkghxhnz       tweet.   positive
9    https co dnd3he3w3n      follow,  positive.


/var/folders/lv/44h1yhq55ld6x4th27m94n4c0000gn/T/ipykernel_12054/1400857903.py:32: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['sentiment'] = df2['sentiment'].str.replace('[', '').str.replace(']', '').str.replace("'","")
/var/folders/lv/44h1yhq55ld6x4th27m94n4c0000gn/T/ipykernel_12054/1400857903.py:33: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df2['word'] = df2['word'].str.replace('[', '').str.replace(']', '').str.replace("'","")


In [ ]:
import json
import pandas as pd
df = pd.read_csv('raw_tweet_data/elonmusk-topic-classification-2023-04-15.csv')
filename = "requests_to_parallel_process_elonmusk_2023-04-17_topic-classification.jsonl"
n_requests = 10000

prompt_prefix = "Classify the following words into one of the following categories: 1. Personal Information 2. Current Events 3. Hobbies and Interests 4. Entertainment 5. Travel 6. Food and Dining 7. Health and Fitness 8. Relationships 9. Work and Career 10. Education 11. Culture and Society 12. Sports and Recreation 13. Personal Growth and Development 14. Fashion and Style 15. Home and Family 16. Technology and Gadgets 17. Art and Creativity 18. Pets and Animals 19. Philosophy and Ethics 20. Nature and Environment Word: "

jobs = df['word'].apply(lambda row: {"model": "text-davinci-003", "prompt": str(prompt_prefix+row) + "\n"})
count = 0
with open(filename, "w") as f:
    for job in jobs:
        # job = job.replace('\x80', '?')
        count = count + 1
        json_string = json.dumps(job)
        f.write(json_string + "\n")
        if(count==n_requests):
            break

In [26]:
# word sentiment class

import json
import pandas as pd
import spacy
import re
count = 0;
clean_data = []
classes =[]
with open(f'parallel_process_results_iamsrk_2023-04-17_topic-classification.jsonl','r') as file:
    for line in file.__iter__():
        errorFound = False
        count = count +1
        if(count == 10100):
            break
        text = line.strip()
        raw_json = json.loads(text)
        # print(count)
        # print(raw_json)
        for item in raw_json[1]:
            if 'error' in item:
                # print(item['error']['message'])
                errorFound = True
        if errorFound:
            continue
        df = pd.DataFrame(raw_json)
        sentence = df['prompt'][0]
        # print(sentence)
        match = re.search(r'Word: (.*?)\n', sentence)
        if match:
            extracted_text = match.group(1)
        clean = extracted_text
        # print(clean)
        clean_data.append(clean)
#        extract topic and sentiment
        topic = df['choices'][1]
        topicJson = topic[0]
        sentence = topicJson['text']
        # print(sentence)
        words = sentence.strip()
        classes.append(words)
        
data2 = {'word':clean_data,'class':classes}
df2 = pd.DataFrame(data2) 
print('topic--class df')
print(df2.shape)
print(df2.head())

df1 = pd.read_csv('raw_tweet_data/iamsrk-topic-classification-2023-04-15-with-sentiment.csv')
print(df1.shape)
print(df1.head())

merged_df = pd.merge(df1,df2, left_on='word',right_on='word',how='right')
print('1st merged-df')
print(merged_df.shape)
print(merged_df.head())
merged_df.to_csv('raw_tweet_data/iamsrk-topic-classification-2023-04-15-merged-with-sentiment.csv', index=False)

topic--class df
(604, 2)
              word                            class
0            Watch           Technology and Gadgets
1    Inspirational  Personal Growth and Development
2               At                   Current Events
3          Showing                Fashion and Style
4  Congratulations                    Relationships
(605, 2)
              word       sentiment
0         Remember         Believe
1          Believe         Believe
2  Congratulations         Believe
3       Maybelline  Congratulatory
4           Credit  Congratulatory
1st merged-df
(604, 3)
              word     sentiment                            class
0            Watch         Watch           Technology and Gadgets
1    Inspirational  Appreciation  Personal Growth and Development
2               At         Watch                   Current Events
3          Showing  Appreciation                Fashion and Style
4  Congratulations       Believe                    Relationships


In [43]:
import pandas as pd
df = pd.read_json('raw_tweet_data/result_class_cnn.jsonl', lines=True)
df.to_csv('parsed/result_class_cnn.csv', index=False)

df = pd.read_json('raw_tweet_data/result_class_justin.jsonl', lines=True)
df.to_csv('parsed/result_class_justin.csv', index=False)

df = pd.read_json('raw_tweet_data/result_class_kamla.jsonl', lines=True)
df.to_csv('parsed/result_class_kamla.csv', index=False)


df = pd.read_json('raw_tweet_data/result_class_messi.jsonl', lines=True)
df.to_csv('parsed/result_class_messi.csv', index=False)

df = pd.read_json('raw_tweet_data/result_class_modi.josnl', lines=True)
df.to_csv('parsed/result_class_modi.csv', index=False)

In [3]:
import json
import pandas as pd
import spacy
import re

def merge_word_sentiment_class(path_topic_class_jsonl,path_topic_class_sentiment,path_word_sentiment_class):
    # word sentiment class
    count = 0;
    clean_data = []
    classes =[]
    with open(path_topic_class_jsonl,'r') as file:
        for line in file.__iter__():
            errorFound = False
            count = count +1
            if(count == 10100):
                break
            text = line.strip()
            raw_json = json.loads(text)
            # print(count)
            # print(raw_json)
            for item in raw_json[1]:
                if 'error' in item:
                    # print(item['error']['message'])
                    errorFound = True
            if errorFound:
                continue
            df = pd.DataFrame(raw_json)
            sentence = df['prompt'][0]
            # print(sentence)
            match = re.search(r'Word: (.*?)\n', sentence)
            if match:
                extracted_text = match.group(1)
            clean = extracted_text
            # print(clean)
            clean_data.append(clean)
    #        extract topic and sentiment
            topic = df['choices'][1]
            topicJson = topic[0]
            sentence = topicJson['text']
            # print(sentence)
            words = sentence.strip()
            classes.append(words)

    data2 = {'word':clean_data,'class':classes}
    df2 = pd.DataFrame(data2) 
    print('topic--class df')
    print(df2.shape)
    print(df2.head(2))

    df1 = pd.read_csv(path_topic_class_sentiment)
    print(df1.shape)
    print(df1.head(2))

    merged_df = pd.merge(df1,df2, left_on='word',right_on='word',how='right')
    print('1st merged-df')
    print(merged_df.shape)
    print(merged_df.head())
    merged_df.to_csv(p_word_sentiment_class, index=False)
    
p_topic_class = 'parallel_process_results_iamsrk_2023-04-17_topic-classification.jsonl'
p_topic_sentiment = 'raw_tweet_data/iamsrk-topic-classification-2023-04-15-with-sentiment.csv'
p_word_sentiment_class = 'posts/iamsrk_word_sentiment_class.csv'
merge_word_sentiment_class(p_topic_class,p_topic_sentiment,p_word_sentiment_class)

topic--class df
(604, 2)
            word                            class
0          Watch           Technology and Gadgets
1  Inspirational  Personal Growth and Development
(605, 2)
       word sentiment
0  Remember   Believe
1   Believe   Believe
1st merged-df
(604, 3)
              word     sentiment                            class
0            Watch         Watch           Technology and Gadgets
1    Inspirational  Appreciation  Personal Growth and Development
2               At         Watch                   Current Events
3          Showing  Appreciation                Fashion and Style
4  Congratulations       Believe                    Relationships


In [52]:
import json
import pandas as pd
import spacy
import re

def merge_word_sentiment_class_csv(p_psot_topic_sentiment_class,p_word_class,p_to_save):
    # word sentiment class


    df1 = pd.read_csv(p_psot_topic_sentiment_class)
    print(df1.shape)
    print(df1.head(2))

    df2 = pd.read_csv(p_word_class)
    print(df2.shape)
    print(df2.head(2))

    merged_df = pd.merge(df1,df2, left_on='word',right_on='word',how='right')
    print('1st merged-df')
    print(merged_df.shape)
    print(merged_df.head())
    merged_df.to_csv(p_to_save)
    
p_psot_topic_sentiment_class = 'parsed/barak_post_topic_sentiment.csv'
p_word_class = 'parsed/result_class_barak.csv'
p_to_save = 'parsed/post_topic_sentiment_class_barak.json'
merge_word_sentiment_class_csv(p_psot_topic_sentiment_class,p_word_class,p_to_save)

(279, 3)
                                                post        word sentiment
0  sister maya chance speak talented asia pacific...     action,  positive
1  sister maya chance speak talented asia pacific...  important,  positive
(329, 2)
        word            class
0     action        Education
1  important  Work and Career
1st merged-df
(935, 4)
                                                post        word sentiment  \
0                                                NaN      action       NaN   
1                                                NaN   important       NaN   
2                                                NaN  leadership       NaN   
3  rise antisemitic attacks alarm us antisemitism...        rise     alarm   
4  rise antisemitic attacks alarm us antisemitism...       alarm     alarm   

                             class  
0                        Education  
1                  Work and Career  
2                  Work and Career  
3  Personal Growth and Deve

In [9]:
import pandas as pd
df = pd.read_csv('raw_tweet_data/elonmusk-tweets-with-topics-sentiment-name-2023-04-15.csv')
df.info()
selected_columns = ['rawContent','topic']
ndf = df.loc[:,selected_columns].copy()
ndf.head()
ndf.to_csv('train/train_con_top_emusk.csv',index=False)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2606 entries, 0 to 2605
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   date            2606 non-null   object 
 1   id              2606 non-null   int64  
 2   rawContent      2606 non-null   object 
 3   retweetedTweet  0 non-null      float64
 4   quotedTweet     88 non-null     object 
 5   hashtags        0 non-null      float64
 6   replyCount      2606 non-null   int64  
 7   retweetCount    2606 non-null   int64  
 8   likeCount       2606 non-null   int64  
 9   quoteCount      2606 non-null   int64  
 10  viewCount       2606 non-null   int64  
 11  clean           2606 non-null   object 
 12  sentiment       2606 non-null   object 
 13  topic           2606 non-null   object 
 14  names           365 non-null    object 
dtypes: float64(2), int64(6), object(7)
memory usage: 305.5+ KB
